In [9]:
import heapq

class Graph:
    def __init__(self, floors, times):
        self.nodes = set() # (ascensor, piso)
        self.edges = dict() # lista ady
        self.unique_vertex = dict() # floor -> [ascensor1, ascensor2, ...]

        for i in range(len(floors)):
            last = None
            for floor in floors[i]:
                # Agrego nodo al grafo e inicializo lista de adyacencia para ese nodo
                self.nodes.add((i, floor))
                self.edges[(i, floor)] = []

                # Agrego aristas entre los nodos del mismo ascensor
                if last != None:
                    self.edges[last].append(((i,floor), times[i])) # Edge en lista de ady: ((ascensor1, piso1), tiempoAscensor1)
                    self.edges[(i,floor)].append((last, times[i])) # Edge en lista de ady: ((ascensor2, piso2), tiempoAscensor2)
                last = (i, floor)
                
                # Agrego aristas ente los nodos de la misma planta
                if floor in self.unique_vertex:
                    for j in self.unique_vertex[floor]:
                        self.edges[(i, floor)].append(((j, floor), 1))
                        self.edges[(j, floor)].append(((i, floor), 1))
                if floor not in self.unique_vertex:
                    self.unique_vertex[floor] = []
                self.unique_vertex[floor].append(i)

    def dijkstra(self, start):
        # Algoritmo de Dijkstra para encontrar el camino más corto desde el nodo dado    
        # usando una cola de prioridad
        queue = []
        heapq.heappush(queue, (0, start))        
        visited = set()
        dist = {vertex: float('infinity') for vertex in self.nodes}
        dist[start] = 0
        while queue:
            current_distance, current_vertex = heapq.heappop(queue)
            if current_vertex in visited:
                continue
            visited.add(current_vertex)
            for neighbor, weight in self.edges[current_vertex]:
                distance = current_distance + weight
                if distance < dist[neighbor]:
                    dist[neighbor] = distance
                    heapq.heappush(queue, (distance, neighbor))
        return dist

In [15]:
def solve(n,k,times,floors):
    # Armamos el grafo
    G = Graph(floors, times)    

    # Agregamos un nodo fantasma con aristas a todos los nodos de la planta 0
    G.nodes.add((-1, -1))
    G.edges[(-1, -1)] = []
    for asc in G.unique_vertex[0]:
        G.edges[(-1, -1)].append(((asc,0),0))

    # Corremos el algoritmo de Dijkstra desde el nodo fantasma
    dist = G.dijkstra((-1, -1))

    # Buscamos el nodo con el menor tiempo
    minSeg = float('infinity')
    for asc in G.unique_vertex[k]:
        if dist[(asc,k)] < minSeg:
            minSeg = dist[(asc,k)]
    if minSeg == float('infinity'):
        return "IMPOSSIBLE", G, dist
    return minSeg, G, dist

floors = [[0, 1, 3, 5, 7, 9, 11, 13, 15, 20, 99],
          [4, 13, 15, 19, 20, 25, 30]]
times = [10, 5]

m,G,dist = solve(2, 30, times, floors)

In [16]:
m

96

In [17]:
G.edges

{(0, 0): [((0, 1), 10)],
 (0, 1): [((0, 0), 10), ((0, 3), 10)],
 (0, 3): [((0, 1), 10), ((0, 5), 10)],
 (0, 5): [((0, 3), 10), ((0, 7), 10)],
 (0, 7): [((0, 5), 10), ((0, 9), 10)],
 (0, 9): [((0, 7), 10), ((0, 11), 10)],
 (0, 11): [((0, 9), 10), ((0, 13), 10)],
 (0, 13): [((0, 11), 10), ((0, 15), 10), ((1, 13), 1)],
 (0, 15): [((0, 13), 10), ((0, 20), 10), ((1, 15), 1)],
 (0, 20): [((0, 15), 10), ((0, 99), 10), ((1, 20), 1)],
 (0, 99): [((0, 20), 10)],
 (1, 4): [((1, 13), 5)],
 (1, 13): [((1, 4), 5), ((0, 13), 1), ((1, 15), 5)],
 (1, 15): [((1, 13), 5), ((0, 15), 1), ((1, 19), 5)],
 (1, 19): [((1, 15), 5), ((1, 20), 5)],
 (1, 20): [((1, 19), 5), ((0, 20), 1), ((1, 25), 5)],
 (1, 25): [((1, 20), 5), ((1, 30), 5)],
 (1, 30): [((1, 25), 5)],
 (-1, -1): [((0, 0), 0)]}

In [18]:
dist

{(0, 5): 30,
 (0, 11): 60,
 (0, 20): 87,
 (1, 15): 76,
 (-1, -1): 0,
 (1, 30): 96,
 (0, 99): 97,
 (0, 1): 10,
 (0, 7): 40,
 (0, 13): 70,
 (1, 20): 86,
 (0, 0): 0,
 (0, 3): 20,
 (0, 9): 50,
 (1, 4): 76,
 (1, 13): 71,
 (0, 15): 77,
 (1, 19): 81,
 (1, 25): 91}